# Notebook: Convert Raw Table Sheet Annotations to txt

## Packages

In [1]:
import pandas as pd
import re

## Code

In [2]:
path_input = 'input.csv' # Diese Stelle austauschen je nachdem, welche csv konvertiert werden soll
path_output = './'
df = pd.read_csv(path_input)[0:3700]

In [3]:
df

,text,tag_1,tag_2,tag_3,tag_4,tag_5,tag_6,tag_7,tag_8,tag_9,tag_10,tag_11,tag_12,tag_13,tag_14,tag_15,tag_16
0,Gemütliche Atmung und rustikale Deko an Wänden.,"Atmung, amb, pos, Gemütliche","Deko an wänden, amb, pos, rustikale",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Zum Servicepersonal, der Küche und dem „Chef i...","Servicepersonal, serv, pos, umsichtig","Chef im ring, serv, pos, umsichtig","Küche, foos, pos, umsichtig","Servicepersonal, serv, pos, freundlich","Chef im ring, serv, pos, freundlich","Küche, serv, pos, freundlich","Servicepersonal, serv, pos, gut eingestimmt","Chef im ring, serv, pos, gut eingestimmt","Küche, foodq, pos, freundlich, gut eingestimmt","Biergarten, restaurantg, pos, viel Freude",NaN,NaN,NaN,NaN,NaN,NaN
2,Der Burger total durchgebraten und trocken.,"Burger, foodq, neg, durchgebraten","Burger, foodq, neg, trocken",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Wie kann ein Betrieb der uns immer gut gefalle...,"Betrieb, restaurantg, neg, 3 klassig",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Das Ambiente ist charmant, das Essen köstlich ...","Ambiente, amb, pos, charmant","Essen, foodq, pos, köstlich","Personal, serv, pos, freundlich",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2149,Wir hatten zwei wunderschöne Abende und alles ...,"NULL, restaurantg, pos,",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2150,Generell eine schöne Erfahrung.,"NULL, restaurantg, pos,",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2151,Wir werden nie wieder eine Filiale dieser Gast...,"Gastronomie Kette, restaurantg, neg,","Personal, serv, neg,",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2152,Wir kommen jederzeit gern wieder.,"NULL, restaurantg, pos,",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:

# 'location general', 'food prices', 'food quality', 'food general',
#     'ambience general', 'service general', 'restaurant prices',
#     'drinks prices', 'restaurant miscellaneous', 'drinks quality',
#     'drinks style_options', 'restaurant general', 'food style_options'

AC = {
    "loca": "location general",
    "serv": "service general",
    "foodq": "food quality",
    "foodg": "food general",
    "amb": "ambience general",
    "drinkq": "drinks quality",
    "restaurantp": "restaurant prices",
    "foodp": "food prices",
    "drinkp": "drinks prices",
    "restaurantm": "restaurant miscellaneous",
    "drinks": "drinks style_options",
    "foods": "food style_options",
    "restaurantg": "restaurant general"
}


def convert_aspects(aspects_raw, text):
    aspects = []
    for aspect in aspects_raw:

        if aspect[0] not in text and aspect[0] != "NULL":
            print(
                f"##### ERROR: Aspect term '{aspect[0]}' not found in text:", text)
        if aspect[1] not in AC.keys():
            print(f"##### ERROR: Aspect category {aspect} not found:", text)
        if aspect[2] not in ["positive", "negative", "neutral"]:
            print(
                f"##### ERROR: Aspect sentiment {aspect[2]} not found in:", text)
        if aspect[3] not in text:
            print(
                f"##### ERROR: Opinion term '{aspect[3]}' not found in text:", text)

        aspects.append([aspect[0], AC[aspect[1]], aspect[2], aspect[3]])
    return aspects

In [5]:
data = ""

total_empty = 0
total_anonym = 0
total_more_than_one = 0
total_more_than_one_sentence = 0
total_examples = 0
total_valid_examples = 0
total_not_english = 0
total_can_t_be_understood = 0


for index, row in df.iterrows():
    rows = row.tolist()
    entry_text = rows[0]
    
    
    all_raw_lists = []
    for aspect_text in rows[2:16]:
        try:
            raw_list = [s.strip() for s in re.split(r',\s*(?!\d)', aspect_text)]
            all_raw_lists.append(raw_list)
        except:
            pass

    # 1. Count all examples without aspects
    no_aspects = False
    if len(all_raw_lists) == 0:
        no_aspects = True
        total_empty += 1

    # 2. Count all examples that are not anoymized
    anonym_found = False
    for sublist in all_raw_lists:
        if "anonym" in sublist:
            if not anonym_found:
                total_anonym += 1
                anonym_found = True
            break
        
    # 3. Count all examples that are not english
    not_english_found = False
    for sublist in all_raw_lists:
        if "not english" in sublist:
            if not not_english_found:
                total_not_english += 1
                not_english_found = True
            break

    # 4. Count all examples with more than one sentence
    more_than_one_sentence_found = False
    for sublist in all_raw_lists:
        if "more than 1 sentence" in sublist:
            if not more_than_one_sentence_found:
                total_more_than_one_sentence += 1
                more_than_one_sentence_found = True
            break
        
    # 5. Count all "can't be understood"
    cant_be_understood_found = False
    for sublist in all_raw_lists:
        if "can't be understood" in sublist:
            if not cant_be_understood_found:
                total_can_t_be_understood += 1
                cant_be_understood_found = True
            break

    if (no_aspects or anonym_found or more_than_one_sentence_found or not_english_found or cant_be_understood_found):
        pass
    else:
        # 5. Check for examples with < 4 sentiment elements
        has_four_elements = True
        for sublist in all_raw_lists:
            if len(sublist) < 4:
                print("##### ERROR:", entry_text, sublist)
                has_four_elements = False

        if has_four_elements == True:
            aspects = convert_aspects(all_raw_lists, entry_text)
            data += entry_text + "####" + str(aspects) + "\n"
            total_valid_examples += 1

    # print(all_raw_lists)

    total_examples += 1

print("Empty:", total_empty)
print("Anonymized:", total_anonym)
print("More than 1 sentence:", total_more_than_one_sentence)
print("Valid examples:", total_valid_examples)
print("Not English:", total_not_english)
print("Can't be understood", total_can_t_be_understood)
print("-----")
print(total_examples)

##### ERROR: Aspect term 'Deko an wänden' not found in text: Gemütliche Atmung und rustikale Deko an Wänden.
##### ERROR: Aspect sentiment pos not found in: Gemütliche Atmung und rustikale Deko an Wänden.
##### ERROR: Aspect term 'Chef im ring' not found in text: Zum Servicepersonal, der Küche und dem „Chef im Ring“ bleibt zu sagen: umsichtig, freundlich und gut eingestimmt machten uns die Stunden im Biergarten recht viel Freude!
##### ERROR: Aspect sentiment pos not found in: Zum Servicepersonal, der Küche und dem „Chef im Ring“ bleibt zu sagen: umsichtig, freundlich und gut eingestimmt machten uns die Stunden im Biergarten recht viel Freude!
##### ERROR: Aspect category ['Küche', 'foos', 'pos', 'umsichtig'] not found: Zum Servicepersonal, der Küche und dem „Chef im Ring“ bleibt zu sagen: umsichtig, freundlich und gut eingestimmt machten uns die Stunden im Biergarten recht viel Freude!
##### ERROR: Aspect sentiment pos not found in: Zum Servicepersonal, der Küche und dem „Chef im Ring

KeyError: 'foos'

In [ ]:
with open(path_output+".txt", 'w', encoding='utf-8') as file:
    file.write(data)